# 運用時の過去レースデータ入れ替え用のスクリプト，今までの機能がまとめて入っている

### （具体的に）
HTML_data.ipynb<br>
↓<br>
trans_text_code.ipynb<br>
↓<br>
making_train_data.ipynb<br>
### これらの機能がひとまとめになっている

## 三カ月に一回，このスクリプトを実行してメンテナンス用データの取得を行う

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import requests
from bs4 import BeautifulSoup
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import sys
import codecs
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
#import module.trans_text_code as trans#ロバストじゃないので今回で書き換えた

In [2]:
def get_season_date(now_date):#日付(datetime型)を渡すと，その日付で購買予測を行う際に使用するデータの区間を返す関数
    use_data_year=now_date.year
    if (now_date.month>=1)and(now_date.month<4):
        use_data_month=1
    elif (now_date.month>=4)and(now_date.month<7):
        use_data_month=4
    elif (now_date.month>=7)and(now_date.month<10):
        use_data_month=7
    elif (now_date.month>=10):
        use_data_month=10
    else:
        print('what???????')
    use_data_date=datetime.datetime(year=use_data_year, month=use_data_month,day=1)
    return use_data_date

def del_file(f_path):#受取ったパスにファイルがあれば削除して新ファイル生成，なければファイル生成だけ行う
    if os.path.exists(f_path)==False:
        pass
    else:
        os.remove('{f_path}'.format(f_path=f_path))
    return None
    
def making_HTML_txt(now_ym):#現在の日付からクローリングを行って学習データを作成する関数,前年分の学習データが存在する前提で実行（今の年度分しか行ってくれない．）
    print('HTMLのテキストファイルののためにクローリング中========================================================')
    data_ym=get_season_date(now_ym)#区間の始まり日付を取得する
    html_ym=data_ym- datetime.timedelta(days=1)#HTML収集時用の日付データ，区間の一日前までのHTMLを取得する（年マタギ防止時の問題防止のため）
    print('now_ym(今日の日付):',now_ym)
    print('data_ym(メンテナンス区間開始日の日付):',data_ym)
    print('HTML_ym(クローリング最終日の日付)',html_ym)
    for place in tqdm(place_master.items()):
        place_num=place[0]
        place_name=place[1]
        url_list=[]
        year=html_ym.year
        txt_months=['01','02','03','04','05','06','07','08','09','10','11','12']
        months=txt_months[:html_ym.month]
        days=['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31']
        txt_f_path="../../bot_database/{name}/{name}_result_txt/{year}_result_{name}.txt".format(year=year,name=place_name)
        del_file(txt_f_path)#前のメンテナンス時の区間のものがあったら一回消す(追記モードのため，消さないとおかしくなってしまう)
        for month in months:
            for day in days:
                url="http://www1.mbrace.or.jp/od2/K/{year}{month}/{place_num}/{day}.html".format(year=year,month=month,place_num=place_num,day=day)

                #print('スクレイピング中',url)
                HTML_res=requests.get(url)
                HTML_res.encoding = HTML_res.apparent_encoding  # この行を追加
                file_path="../../bot_database/{name}/{name}_result_txt/{year}_result_{name}.txt".format(year=year,name=place_name)

                if HTML_res.status_code==200:#URLが存在するときのみスクレイピングを行う。
                    HTML_doc=HTML_res.content#pip install chardet 要　文字化け防止
                    date=datetime.date(int(year),int(month),int(day))
                    #HTML=BeautifulSoup(HTML_doc,'lxml')#pip install lxmlでインストール要
                    HTML=BeautifulSoup(HTML_doc,'html.parser')#pip install lxmlでインストール要
                    result_data_text=[]
                    for HTML in HTML.find_all('pre'):
                        result_data_text.append(HTML.get_text())
                    result_data_text[0]='\n'
                    with open(file_path,'a') as f:
                        for i in range(len(result_data_text)):
                            if(i==1):
                                print('date:{}\n'.format(date),file=f)
                                print(result_data_text[i][:924],file=f)
                            else:
                                print(result_data_text[i][:924],file=f)
                                print('\n',file=f)
                    time.sleep(2)
                else:
                     pass
    return None

def result_text_trans(now_ym):#HTMLのテキストファイルの文字コードをutf-8にまとめて変換する関数
    print('HTMLのテキストファイルの文字コードを変換中========================================================')
    place_master=master.get_place_master()
    for place in place_master.items():
        place_name=place[1]
        dir_path = "../../bot_database/{place_name}/{place_name}_result_txt_utf8/".format(place_name=place_name)
        if os.path.exists(dir_path)==False:
            os.makedirs(dir_path)
        else:
            pass
    #テキストファイルの文字コードの変換==========================================================
    data_ym=get_season_date(now_ym)#区間の始まり日付を取得する
    html_ym=data_ym- datetime.timedelta(days=1)#HTML収集時用の日付データ，区間の一日前までのHTMLを取得する（年マタギ防止時の問題防止のため）
    print('now_ym(今日の日付):',now_ym)
    print('HTML_ym(クローリング最終日の日付)',html_ym)
    end_year=html_ym.year
    years=['{}'.format(year) for year in np.arange(2012,end_year+1)]#エンコード方式を変更する元テキストファイルがある年度(クローリング結果)
    for place in tqdm(place_master.items()):
        place_name=place[1]
        url_list=[]
        for year in years:
            read_fpath="../../bot_database/{place_name}/{place_name}_result_txt/{year}_result_{place_name}.txt".format(place_name=place_name,year=year)
            write_fpath="../../bot_database/{place_name}/{place_name}_result_txt_utf8/{year}_result_{place_name}_utf8.txt".format(place_name=place_name,year=year)
            try:
                sf = codecs.open(read_fpath, 'r', encoding='shift-jis')
                uf = codecs.open(write_fpath, 'w', encoding='utf-8')
                for line in sf:
                    uf.write(line)
                sf.close()
                uf.close()
            except FileNotFoundError:
                print('file_not_found_error!!!!!!!!!!',place_name,year)
    return None

def result_csv_make(now_ym):#HTMLのテキストファイルから必要な情報をぬきだして　csvに変換する関数
    #ディレクトリを作っておく
    print('HTMLのテキストファイルからresult_csvを製作中========================================================')
    data_ym=get_season_date(now_ym)#区間の始まり日付を取得する
    html_ym=data_ym- datetime.timedelta(days=1)#HTML収集時用の日付データ，区間の一日前までのHTMLを取得する（年マタギ防止時の問題防止のため）
    print('now_ym(今日の日付):',now_ym)
    print('HTML_ym(クローリング最終日の日付)',html_ym)
    end_year=html_ym.year
    years=['{}'.format(year) for year in np.arange(2012,end_year+1)]#エンコード方式を変更する元テキストファイルがある年度(クローリング結果)
    place_master=master.get_place_master()
    for place in place_master.items():
        place_name=place[1]
        dir_path = "../../bot_database/{place_name}/{place_name}_result_csv/".format(place_name=place_name)
        if os.path.exists(dir_path)==False:
            os.makedirs(dir_path)
        else:
            pass
    for place in tqdm(place_master.items()):
        place_name=place[1]
        #print(place_name,'\n')
        #years=['2012','2013','2014','2015','2016','2017','2018','2019','2020']
        place=place_name
        date='2222-22-22'
        racer_1 = 0  # 同着レースを見つけるための印付け
        racer_2 = 0  # 同着レースを見つけるための印付け
        racer_3 = 0  # 同着レースを見つけるための印付け
        for year in years:
            train_data_df=pd.DataFrame(columns=['date','result_com','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID'])
            #read_resultdata_path = "{}_result_txt/{}_result_{}.txt".format(place,year,place)
            read_resultdata_path = "../../bot_database/{place_name}/{place_name}_result_txt_utf8/{year}_result_{place_name}_utf8.txt".format(place_name=place_name,year=year)
            #write_file_path="{}_result_csv2/{}_result_{}.csv".format(place,year,place)
            write_file_path="../../bot_database/{place_name}/{place_name}_result_csv/{place_name}_result_{year}.csv".format(year=year,place_name=place_name)
            try:
                resultdata = open(read_resultdata_path, "r", encoding="utf-8_sig")
                money=0
                normal_race=0
                get_money=0
                for line in resultdata:
                # print(line[:12])
                # print(line[4:5]) #レースを示す"R"の入っている位置
                # print(line[3:4])#レースの番号が入っている位置
                # print(line[2:4])#着の情報が入っている位置
                # print(line[6:7])#艇番号の入っている位置
                # print(line[8:12])#登録暗号４桁の入っている位置
                # ////////////////////////////////以下レース番号の取得
                    if line[0]=='d':
                        date=line[5:15]
                    if line[4:5] == "R":
                        if line[2:4]=="10":
                            number_race=line[2:4]
                        elif line[2:4]=="11":
                            number_race=line[2:4]
                        elif line[2:4]=="12":
                            number_race=line[2:4]
                        else:
                            number_race = line[3:4]
                        racer_1 = 0  # 同着レースを見つけるための印付け
                        racer_2 = 0  # 同着レースを見つけるための印付け
                        racer_3 = 0  # 同着レースを見つけるための印付け
            # //////////////////////////////以下着順獲得
                # racer_1st=0#同着レースを見つけるための印付け
                # racer_2st=0#同着レースを見つけるための印付け
                    if line[2:4] == "01":
                        racer_1 = int(line[6:7])
                    elif line[2:4] == "02":
                        racer_2 = int(line[6:7])
                    elif line[2:4] == "03":
                        racer_3 = int(line[6:7])

            # //////////////////////////////////以下選手の登録番号取得
                    if line[6:7] == "1":
                        racer_1_ID = int(line[8:12])
                        racer_1_b=int(line[22:24])#ボート番号
                        racer_1_m=int(line[27:29])#モータ番号
                    elif line[6:7] == "2":
                        racer_2_ID = int(line[8:12])
                        racer_2_b=int(line[22:24])
                        racer_2_m=int(line[27:29])
                    elif line[6:7] == "3":
                        racer_3_ID = int(line[8:12])
                        racer_3_b=int(line[22:24])
                        racer_3_m=int(line[27:29])
                    elif line[6:7] == "4":
                        racer_4_ID = int(line[8:12])
                        racer_4_b=int(line[22:24])
                        racer_4_m=int(line[27:29])
                    elif line[6:7] == "5":
                        racer_5_ID = int(line[8:12])
                        racer_5_b=int(line[22:24])
                        racer_5_m=int(line[27:29])
                    elif line[6:7] == "6":
                        racer_6_ID = int(line[8:12])
                        racer_6_b=int(line[22:24])
                        racer_6_m=int(line[27:29])
            # //////////////////以下着順の組み合わせの選別
                    if racer_1!=0 and racer_2!=0 and racer_3!=0: 
                        if line[2:4] == "06":
                            racers_arr=['1','2','3','4','5','6']
                            result_com=0
                            result_com+=(racer_1 - 1)*20
                            #racers_arr=racers_arr.remove(str(racer_1))
                            racers_arr.remove(str(racer_1))
                            index_2=racers_arr.index(str(racer_2))
                            result_com+=index_2*4
                            #racers_arr=racers_arr.remove(str(racer_2))
                            racers_arr.remove(str(racer_2))
                            index_3=racers_arr.index(str(racer_3))
                            result_com+=(index_3+1)
            # /////////////////////////////////以下取り出した情報をdataframe化
                 #レースの配当金の情報を追加する
                    if line[8:9] == "３":
                        if get_money==0:
                            money=line[23:29]
                            try:
                                money=int(money)
                                get_money=1
                            except ValueError:
                                print('Error')
                                get_money=0
                # /////////////////////////////////以下取り出した情報をdataframe化
                    if line[2:4] == "06":#フライング、事故が発生したレースを除外するために6(全員着を得ている)があるときのみ書き込み
                        normal_race=1
                        money=0
                        get_money=0
                    if (money!=0) and (normal_race==1):#フライング、事故が発生したレースを除外するために6(全員着を得ている)があるときのみ書き込み
                        add_dict = {'date':date,"result_com": result_com,"money":money, "number_race": number_race, "racer_1_ID": racer_1_ID, "racer_2_ID": racer_2_ID,"racer_3_ID": racer_3_ID, "racer_4_ID": racer_4_ID, "racer_5_ID": racer_5_ID, "racer_6_ID": racer_6_ID,'racer_1_b':racer_1_b,'racer_1_m':racer_1_m,'racer_2_b':racer_2_b,'racer_2_m':racer_2_m,'racer_3_b':racer_3_b,'racer_3_m':racer_3_m,'racer_4_b':racer_4_b,'racer_4_m':racer_4_m,'racer_5_b':racer_5_b,'racer_5_m':racer_5_m,'racer_6_b':racer_6_b,'racer_6_m':racer_6_m}
                        add_df = pd.DataFrame(add_dict, index=[''])
                        train_data_df = train_data_df.append(add_df)
                        money=0
                        normal_race=0
                #//////////////////////////////////////CSVファイルへ出力
                train_data_df.to_csv(write_file_path)
            except FileNotFoundError:
                print('file_not_found_error!!!!!!!!!!',place_name,year)
    return None

def train_csv_make(now_ym):#今の区間までのhtmlから作成したレースデータcsvに別でダウンロードした半期の選手データを結合する,選手データは手動で配置
    #ディレクトリを作っておく
    print('result_csvと結合してtrainデータを製作中========================================================')
    data_ym=get_season_date(now_ym)#区間の始まり日付を取得する
    html_ym=data_ym- datetime.timedelta(days=1)#HTML収集時用の日付データ，区間の一日前までのHTMLを取得する（年マタギ防止時の問題防止のため）
    print('now_ym(今日の日付):',now_ym)
    print('HTML_ym(クローリング最終日の日付)',html_ym)
    end_year=html_ym.year
    years=['{}'.format(year) for year in np.arange(2012,end_year+1)]#エンコード方式を変更する元テキストファイルがある年度(クローリング結果)
    place_master=master.get_place_master()
    for place in tqdm(place_master.items()):
        place_name=place[1]
        dir_path = "../../bot_database/{place_name}/{place_name}_train/".format(place_name=place_name)
        if os.path.exists(dir_path)==False:
            os.makedirs(dir_path)
        else:
            pass

        dir_path2 = "../../bot_database/{place_name}/{place_name}_train/train_by_year/".format(place_name=place_name)
        if os.path.exists(dir_path2)==False:
            os.makedirs(dir_path2)
        else:
            pass
        train_data_df=pd.DataFrame(columns=['date','result_com','money','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
        #years=["12","13","14","15","16","17","18","19","20"]
        para_year=['{}'.format(year)[-2:]for year in np.arange(2012,end_year+1)]#パラメータシート読み込み用に年度を加工（下二桁）
        place=place_name
        write_train_file_path="../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)
        for year in para_year:
            #使用するファイルの定義
            para_file_path="../../bot_database/racer_para/{year}/{year}.csv".format(year=year)
            result_file_path="../../bot_database/{place_name}/{place_name}_result_csv/{place_name}_result_20{year}.csv".format(place_name=place_name,year=year)
            write_train_by_year_file_path="../../bot_database/{place_name}/{place_name}_train/train_by_year/train_20{year}_{place_name}.csv".format(place_name=place_name,year=year)#作成したデータの書き込み先
            print(year)
            try:
                #/////////////////////////////////////////////以下データフレームの作成
                train_data_by_year_df=pd.DataFrame(columns=['date','result_com','money','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
                para_df=pd.read_csv(para_file_path)
                para_df=para_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
                result_df=pd.read_csv(result_file_path)
                result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
                display(para_df)
                ####==========================================
                resultdata = open(result_file_path, "r", encoding="utf-8_sig")
                money=0
                normal_race=0
                get_money=0
                for index,series in result_df.iterrows():
                    add_df=pd.DataFrame(columns=['date','result_com','money','number_race','racer_1_ID','racer_2_ID','racer_3_ID','racer_4_ID','racer_5_ID','racer_6_ID','racer_1_rank','racer_1_male','racer_1_age','racer_1_doub','racer_1_ave_st','racer_2_rank','racer_2_male','racer_2_age','racer_2_doub','racer_2_ave_st','racer_3_rank','racer_3_male','racer_3_age','racer_3_doub','racer_3_ave_st','racer_4_rank','racer_4_male','racer_4_age','racer_4_doub','racer_4_ave_st','racer_5_rank','racer_5_male','racer_5_age','racer_5_doub','racer_5_ave_st','racer_6_rank','racer_6_male','racer_6_age','racer_6_doub','racer_6_ave_st'])
                    #///////////////////////////////////////レースに出ているレーサーの成績を検索＆取得
                    ID_1=series['racer_1_ID']
                    ID_2=series['racer_2_ID']
                    ID_3=series['racer_3_ID']
                    ID_4=series['racer_4_ID']
                    ID_5=series['racer_5_ID']
                    ID_6=series['racer_6_ID']
                    racer_1_df=para_df[para_df['racer_ID']==ID_1]
                    racer_2_df=para_df[para_df['racer_ID']==ID_2]
                    racer_3_df=para_df[para_df['racer_ID']==ID_3]
                    racer_4_df=para_df[para_df['racer_ID']==ID_4]
                    racer_5_df=para_df[para_df['racer_ID']==ID_5]
                    racer_6_df=para_df[para_df['racer_ID']==ID_6]
                    try:
                        if series['result_com']==40:
                            pass#同着のデータは削除
                    #追加していくデータフレームを作成
                        else:
                            add_df= pd.DataFrame({'date':series['date'],
                                                    'result_com':series['result_com'],
                                                    'money':series['money'],
                                                    'number_race':series['number_race'],
                                                    'racer_1_ID':series['racer_1_ID'],
                                                    'racer_2_ID':series['racer_2_ID'],
                                                    'racer_3_ID':series['racer_3_ID'],
                                                    'racer_4_ID':series['racer_4_ID'],
                                                    'racer_5_ID':series['racer_5_ID'],
                                                    'racer_6_ID':series['racer_6_ID'],
                                                    'racer_1_bo':series['racer_1_b'],
                                                    'racer_1_mo':series['racer_1_m'],
                                                    'racer_2_bo':series['racer_2_b'],
                                                    'racer_2_mo':series['racer_2_m'],
                                                    'racer_3_bo':series['racer_3_b'],
                                                    'racer_3_mo':series['racer_3_m'],
                                                    'racer_4_bo':series['racer_4_b'],
                                                    'racer_4_mo':series['racer_4_m'],
                                                    'racer_5_bo':series['racer_5_b'],
                                                    'racer_5_mo':series['racer_5_m'],
                                                    'racer_6_bo':series['racer_6_b'],
                                                    'racer_6_mo':series['racer_6_m'],
                                                    'racer_1_rank':racer_1_df.iat[0,1],
                                                    'racer_1_male':racer_1_df.iat[0,2],
                                                    'racer_1_age':racer_1_df.iat[0,3],
                                                    'racer_1_doub':racer_1_df.iat[0,4],
                                                    'racer_1_ave_st':racer_1_df.iat[0,5],
                                                    'racer_2_rank':racer_2_df.iat[0,1],
                                                    'racer_2_male':racer_2_df.iat[0,2],
                                                    'racer_2_age':racer_2_df.iat[0,3],
                                                    'racer_2_doub':racer_2_df.iat[0,4],
                                                    'racer_2_ave_st':racer_2_df.iat[0,5],
                                                    'racer_3_rank':racer_3_df.iat[0,1],
                                                    'racer_3_male':racer_3_df.iat[0,2],
                                                    'racer_3_age':racer_3_df.iat[0,3],
                                                    'racer_3_doub':racer_3_df.iat[0,4],
                                                    'racer_3_ave_st':racer_3_df.iat[0,5],
                                                    'racer_4_rank':racer_4_df.iat[0,1],
                                                    'racer_4_male':racer_4_df.iat[0,2],
                                                    'racer_4_age':racer_4_df.iat[0,3],
                                                    'racer_4_doub':racer_4_df.iat[0,4],
                                                    'racer_4_ave_st':racer_4_df.iat[0,5],
                                                    'racer_5_rank':racer_5_df.iat[0,1],
                                                    'racer_5_male':racer_5_df.iat[0,2],
                                                    'racer_5_age':racer_5_df.iat[0,3],
                                                    'racer_5_doub':racer_5_df.iat[0,4],
                                                    'racer_5_ave_st':racer_5_df.iat[0,5],
                                                    'racer_6_rank':racer_6_df.iat[0,1],
                                                    'racer_6_male':racer_6_df.iat[0,2],
                                                    'racer_6_age':racer_6_df.iat[0,3],
                                                    'racer_6_doub':racer_6_df.iat[0,4],
                                                    'racer_6_ave_st':racer_6_df.iat[0,5] }, index=[''])
                        #//////////////////////////////データフレームにadd_dfを追加していく。
                        train_data_by_year_df=train_data_by_year_df.append(add_df)
                        train_data_df=train_data_df.append(add_df)
                    except:
                        print('CAREFULL!!!!    NOT FOUND RACER ',place,series['date'])
                        print(add_df)
                        
                train_data_by_year_df.to_csv(write_train_by_year_file_path)
            except FileNotFoundError:
                print('file_not_found_error!!!!!!!!!!',place_name,year)
        train_data_df.to_csv(write_train_file_path,encoding="utf-8_sig")
    return None #何も返さない

In [3]:
place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    dir_path = "../../bot_database/{}/{}_result_txt".format(place[1],place[1])
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass

## 以下にすべての機能をまとめる（これをさらに一つの関数にまとめてもいいかも）

In [4]:
today=datetime.date.today()#今日の日付の取得
# print(today,'======================================')
# making_HTML_txt(today)#クローリング，テキストファイルの生成
# result_text_trans(today)#文字コードの変換とutf-8b版の出力
# result_csv_make(today)#HTMLのテキストファイルから必要な情報をぬきだして　csvに変換する関数
#today=datetime.date(2022, 1, 1)#今日の日付の取得
train_csv_make(today)#今の区間までのhtmlから作成したレースデータcsvに別でダウンロードした半期の選手データを結合する,選手データは手動で配置
 

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

result_csvと結合してtrainデータを製作中========================================================
now_ym(今日の日付): 2022-04-19
HTML_ym(クローリング最終日の日付) 2022-03-31 00:00:00
12


,racer_ID,racer_rank,racer_male,racer_age,racer_doub_win,racer_ave_st_time
0,1485.0,2,1,70.0,0.170,0.19
1,1710.0,2,1,69.0,0.310,0.18
2,1759.0,2,1,66.0,0.197,0.23
3,2014.0,3,1,65.0,0.382,0.15
4,2015.0,1,1,65.0,0.106,0.23
...,...,...,...,...,...,...
1594,2998.0,1,1,49.0,0.072,0.22
1595,3223.0,1,1,47.0,0.094,0.21
1596,3271.0,2,1,46.0,0.277,0.18
1597,3402.0,1,1,44.0,0.061,0.16


13


,racer_ID,racer_rank,racer_male,racer_age,racer_doub_win,racer_ave_st_time
0,1485.0,2,1,71.0,0.151,0.19
1,1710.0,2,1,70.0,0.277,0.22
2,1759.0,2,1,67.0,0.183,0.22
3,2014.0,3,1,66.0,0.388,0.14
4,2223.0,2,1,69.0,0.180,0.21
...,...,...,...,...,...,...
1628,3728.0,2,1,41.0,0.314,0.17
1629,3774.0,4,0,38.0,0.464,0.15
1630,3954.0,2,1,36.0,0.220,0.20
1631,4178.0,1,0,29.0,0.208,0.20


...

,racer_ID,racer_rank,racer_male,racer_age,racer_doub_win,racer_ave_st_time
0,2014.0,2,1,74.0,0.275,0.18
1,2538.0,2,1,72.0,0.150,0.21
2,2841.0,2,1,64.0,0.202,0.20
3,2844.0,2,1,64.0,0.202,0.19
4,2876.0,2,1,63.0,0.377,0.15
...,...,...,...,...,...,...
1654,4768.0,1,1,28.0,0.040,0.19
1655,4799.0,2,1,27.0,0.127,0.17
1656,4805.0,2,1,32.0,0.085,0.17
1657,4838.0,2,1,30.0,0.077,0.20


CAREFULL!!!!    NOT FOUND RACER  omura 2021-11-20
Empty DataFrame
Columns: [date, result_com, money, number_race, racer_1_ID, racer_2_ID, racer_3_ID, racer_4_ID, racer_5_ID, racer_6_ID, racer_1_rank, racer_1_male, racer_1_age, racer_1_doub, racer_1_ave_st, racer_2_rank, racer_2_male, racer_2_age, racer_2_doub, racer_2_ave_st, racer_3_rank, racer_3_male, racer_3_age, racer_3_doub, racer_3_ave_st, racer_4_rank, racer_4_male, racer_4_age, racer_4_doub, racer_4_ave_st, racer_5_rank, racer_5_male, racer_5_age, racer_5_doub, racer_5_ave_st, racer_6_rank, racer_6_male, racer_6_age, racer_6_doub, racer_6_ave_st]
Index: []

[0 rows x 40 columns]
CAREFULL!!!!    NOT FOUND RACER  omura 2021-11-21
Empty DataFrame
Columns: [date, result_com, money, number_race, racer_1_ID, racer_2_ID, racer_3_ID, racer_4_ID, racer_5_ID, racer_6_ID, racer_1_rank, racer_1_male, racer_1_age, racer_1_doub, racer_1_ave_st, racer_2_rank, racer_2_male, racer_2_age, racer_2_doub, racer_2_ave_st, racer_3_rank, racer_3_male,

22


,racer_ID,racer_rank,racer_male,racer_age,racer_doub_win,racer_ave_st_time
0,2014.0,2,1,74.0,0.242,0.17
1,2538.0,2,1,72.0,0.168,0.19
2,2841.0,2,1,65.0,0.108,0.20
3,2844.0,2,1,64.0,0.186,0.17
4,2876.0,2,1,64.0,0.253,0.17
...,...,...,...,...,...,...
1623,4751.0,2,1,33.0,0.139,0.19
1624,4767.0,2,0,29.0,0.057,0.17
1625,4820.0,2,1,29.0,0.157,0.18
1626,4996.0,1,1,28.0,0.020,0.21


100%|███████████████████████████████████████████████████████████████████████████████| 24/24 [2:08:49<00:00, 322.04s/it]


## 更新を忘れてしまって前の区間で行いたい場合

In [16]:

today=datetime.date(2022, 1, 1)#今日の日付の取得
print(today,'======================================')
making_HTML_txt(today)#クローリング，テキストファイルの生成
result_text_trans(today)#文字コードの変換とutf-8b版の出力
result_csv_make(today)#HTMLのテキストファイルから必要な情報をぬきだして　csvに変換する関数
train_csv_make(today)#今の区間までのhtmlから作成したレースデータcsvに別でダウンロードした半期の選手データを結合する,選手データは手動で配置

  0%|                                                                                           | 0/24 [00:00<?, ?it/s]

2022-01-01 ======================================
HTMLのテキストファイルののためにクローリング中========================================================
now_ym(今日の日付): 2022-01-01
data_ym(メンテナンス区間開始日の日付): 2022-01-01 00:00:00
HTML_ym(クローリング最終日の日付) 2021-12-31 00:00:00


 67%|██████████████████████████████████████████████████▋                         | 16/24 [9:08:59<4:34:29, 2058.73s/it]


KeyboardInterrupt: 

## 機能作成，チェック

In [5]:
para_year=['{}'.format(year)[-2:]for year in np.arange(2012,end_year+1)]#エンコード方式を変更する元テキストファイルがある年度(クローリング結果)
para_year

NameError: name 'end_year' is not defined

In [ ]:
data_ym=get_season_date(now_ym)#区間の始まり日付を取得する
html_ym=data_ym- datetime.timedelta(days=1)#HTML収集時用の日付データ，区間の一日前までのHTMLを取得する（年マタギ防止時の問題防止のため）
print('now_ym(今日の日付):',now_ym)
print('HTML_ym(クローリング最終日の日付)',html_ym)
end_year=html_ym.year
years=['{}'.format(year) for year in np.arange(2012,end_year+1)]#エンコード方式を変更する元テキストファイルがある年度(クローリング結果)


In [ ]:
#for year in years:
    
    print(year,'年目が終了')
